Будем осуществлять работу с непростым набором данных о состоянии здоровья лошадей, испытывающих кишечные колики. Цель – максимально корректно заполнить пропуски.

**Задание 1.** Загрузка данных

Изучить представленный набор данных на основе описания его столбцов, загрузить его и оставить 8 столбцов для дальнейшего изучения: surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome.

In [91]:
# Литература:
# https://coderoad.ru/29287224/Pandas-%D1%87%D0%B8%D1%82%D0%B0%D1%82%D1%8C-%D0%B2-%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%86%D0%B5-%D0%B1%D0%B5%D0%B7-%D0%B7%D0%B0%D0%B3%D0%BE%D0%BB%D0%BE%D0%B2%D0%BA%D0%BE%D0%B2
# https://coderoad.ru/46397526/%D0%9A%D0%B0%D0%BA-%D0%B8%D1%81%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D0%BE%D0%B2%D0%B0%D1%82%D1%8C-%D0%BE%D0%BF%D1%86%D0%B8%D1%8E-na_values-%D0%B2-%D1%84%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D0%B8-pd-read-csv

import pandas as pd
horse_data = pd.read_csv('horse_data.csv', header=None, usecols=[0,1,3,4,5,6,10,22], names=['surgery?', 'age', 'rectal_temperature', 'pulse', 'respiratory_rate', 'temperature_of_extremities', 'pain', 'outcome'], na_values='?')
horse_data.head()

,surgery?,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0


**Задание 2.** Первичное изучение данных

Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

In [92]:
horse_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery?                    299 non-null    float64
 1   age                         300 non-null    int64  
 2   rectal_temperature          240 non-null    float64
 3   pulse                       276 non-null    float64
 4   respiratory_rate            242 non-null    float64
 5   temperature_of_extremities  244 non-null    float64
 6   pain                        245 non-null    float64
 7   outcome                     299 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.9 KB


* К качественным (порядковым) относятся признаки - 'age', 'temperature_of_extremities'
* К качественным (номинальным) относятся признаки -'surgery?', 'outcome', согласно документации, отнесем также'pain'
* К количественным (непрерывным) признакам относятся - 'rectal_temperature' и к количественным (дискретным) относятся - 'pulse', 'respiratory_rate' для них и посчитаем основные статистики

In [93]:
# min, max, median, mean, 1 и 3 квартили
horse_data[['rectal_temperature', 'pulse', 'respiratory_rate']].describe()

,rectal_temperature,pulse,respiratory_rate
count,240.000000,276.000000,242.000000
mean,38.167917,71.913043,30.417355
std,0.732289,28.630557,17.642231
min,35.400000,30.000000,8.000000
25%,37.800000,48.000000,18.500000
50%,38.200000,64.000000,24.500000
75%,38.500000,88.000000,36.000000
max,40.800000,184.000000,96.000000


In [94]:
# Мода
horse_data[['rectal_temperature', 'pulse', 'respiratory_rate']].mode()

,rectal_temperature,pulse,respiratory_rate
0,38.0,48.0,20.0


In [95]:
# Рахмах
horse_data[['rectal_temperature', 'pulse', 'respiratory_rate']].max()- horse_data[['rectal_temperature', 'pulse', 'respiratory_rate']].min()

rectal_temperature      5.4
pulse                 154.0
respiratory_rate       88.0
dtype: float64

In [96]:
# СКО
horse_data[['rectal_temperature', 'pulse', 'respiratory_rate']].std()

rectal_temperature     0.732289
pulse                 28.630557
respiratory_rate      17.642231
dtype: float64

In [97]:
# Дисперсия
horse_data[['rectal_temperature', 'pulse', 'respiratory_rate']].var()

rectal_temperature      0.536247
pulse                 819.708775
respiratory_rate      311.248328
dtype: float64

In [98]:
# межквартильный размах
Q1 = horse_data[['rectal_temperature', 'pulse', 'respiratory_rate']].quantile(0.25)
Q3 = horse_data[['rectal_temperature', 'pulse', 'respiratory_rate']].quantile(0.75)
IQR = Q3 - Q1
print(Q1)
print(Q3)
IQR

rectal_temperature    37.8
pulse                 48.0
respiratory_rate      18.5
Name: 0.25, dtype: float64
rectal_temperature    38.5
pulse                 88.0
respiratory_rate      36.0
Name: 0.75, dtype: float64


rectal_temperature     0.7
pulse                 40.0
respiratory_rate      17.5
dtype: float64

*Определим выбросы*

Исходим из того, что распределение по признаку "rectal_temperature" нормальное:
мода = 38.0, медиана = 38.200000, mean = 38.167917 - достаточно близки по значению друг к другу, что является признаком нормального распределения.
Воспользыемся методом определения выбросов на основе межквартильного размаха.

Пустые значения "rectal_temperature" не учитываются/удалены при расчете.

In [99]:
lower_bound = Q1 - (1.5 * IQR) 
upper_bound = Q3 + (1.5 * IQR)
remove_outliers = horse_data[horse_data['rectal_temperature'].between(lower_bound[0], upper_bound[0], inclusive=True)].sort_values('rectal_temperature')
remove_outliers

,surgery?,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
113,1.0,1,36.8,60.0,28.0,NaN,NaN,2.0
277,2.0,1,36.9,50.0,40.0,2.0,1.0,1.0
70,2.0,1,37.0,60.0,20.0,3.0,3.0,3.0
292,1.0,1,37.0,66.0,20.0,1.0,4.0,2.0
190,1.0,1,37.1,40.0,8.0,NaN,3.0,1.0
...,...,...,...,...,...,...,...,...
165,1.0,1,39.4,120.0,48.0,NaN,NaN,3.0
231,2.0,1,39.5,92.0,28.0,3.0,5.0,2.0
287,2.0,9,39.5,84.0,30.0,NaN,NaN,1.0
117,2.0,1,39.5,NaN,NaN,3.0,3.0,3.0


In [100]:
# Значения определенные как выбросы
pd.concat([horse_data, remove_outliers]).drop_duplicates(keep=False).dropna().sort_values('rectal_temperature')

,surgery?,age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
44,1.0,1,35.4,140.0,24.0,3.0,4.0,3.0
80,1.0,1,36.4,98.0,35.0,3.0,4.0,2.0
118,1.0,1,36.5,78.0,30.0,1.0,5.0,1.0
298,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0
251,2.0,1,36.6,42.0,18.0,3.0,1.0,2.0
99,2.0,1,39.6,108.0,51.0,3.0,2.0,1.0
20,1.0,1,39.9,72.0,60.0,1.0,5.0,1.0
54,2.0,1,40.3,114.0,36.0,3.0,2.0,3.0
91,2.0,1,40.3,114.0,36.0,3.0,2.0,2.0
259,1.0,1,40.8,72.0,42.0,3.0,2.0,2.0


**Задание 3.**  Работа с пропусками

Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.



In [101]:
# Количество пропусков по столбцам
horse_data.isnull().sum()

surgery?                       1
age                            0
rectal_temperature            60
pulse                         24
respiratory_rate              58
temperature_of_extremities    56
pain                          55
outcome                        1
dtype: int64

In [102]:
# Оставим строки в которых 6 и более заполненных ячеек
horse_data.dropna(thresh=6, inplace=True)
horse_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery?                    277 non-null    float64
 1   age                         278 non-null    int64  
 2   rectal_temperature          238 non-null    float64
 3   pulse                       269 non-null    float64
 4   respiratory_rate            240 non-null    float64
 5   temperature_of_extremities  236 non-null    float64
 6   pain                        238 non-null    float64
 7   outcome                     277 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 19.5 KB


In [103]:
# Показатели протекания болезни могут различаться для лошадей разных возрастных категорий
#  Заполним пустые значения количественных признаков медианным значением, с учетом замеченных зависимостей.
horse_data.groupby(['age'])['rectal_temperature', 'respiratory_rate', 'pulse'].median()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,rectal_temperature,respiratory_rate,pulse
age,,,
1,38.1,24.0,60.0
9,38.6,49.0,124.0


In [104]:
horse_data['rectal_temperature'].fillna(horse_data.groupby(['age'])['rectal_temperature'].transform('median'), inplace=True)
horse_data['respiratory_rate'].fillna(horse_data.groupby(['age'])['respiratory_rate'].transform('median'), inplace=True)
horse_data['pulse'].fillna(horse_data.groupby(['age'])['pulse'].transform('median'), inplace=True)
len(horse_data) - horse_data.count()

surgery?                       1
age                            0
rectal_temperature             0
pulse                          0
respiratory_rate               0
temperature_of_extremities    42
pain                          40
outcome                        1
dtype: int64

In [105]:
def choice_quantile(df, *quants):
    for i, quant in enumerate(quants):
        if df<=quant:
            return i+1
        else:
            continue

In [106]:
# Также обратил внимание, что есть зависимость pulse и temperature_of_extremities (чем выше pulse, тем больше значение temperature_of_extremities)
# temperature_of_extremities имеет 4 категории. Проставим каждую категорию temperature_of_extremities по значению pulse,
# в зависимости от вхождения значения pulse в 1, 2, 3 или 4 квартиль.
quants = pd.DataFrame(horse_data['pulse'].quantile([0.25, 0.5, 0.75, 1]))['pulse'].to_list()
print(quants)
horse_data['temperature_of_extremities'].fillna(horse_data['pulse'].apply(choice_quantile, args=(quants)), inplace=True)

[48.0, 62.0, 88.0, 184.0]


In [107]:
horse_data.isnull().sum()

surgery?                       1
age                            0
rectal_temperature             0
pulse                          0
respiratory_rate               0
temperature_of_extremities     0
pain                          40
outcome                        1
dtype: int64

In [108]:
# заполним surgery? и outcome значениями моды
horse_data['surgery?'].fillna(horse_data['surgery?'].mode()[0], inplace=True)
horse_data['outcome'].fillna(horse_data['outcome'].mode()[0], inplace=True)
horse_data.isnull().sum()

surgery?                       0
age                            0
rectal_temperature             0
pulse                          0
respiratory_rate               0
temperature_of_extremities     0
pain                          40
outcome                        0
dtype: int64

In [109]:
horse_data['pain'].value_counts()

3.0    67
2.0    58
5.0    39
1.0    37
4.0    37
Name: pain, dtype: int64

In [110]:
# зависимостей для восстановления pain не вижу, 
# заполним pain значениями моды
horse_data['pain'].fillna(horse_data['pain'].mode()[0], inplace=True)
horse_data.isnull().sum()

surgery?                      0
age                           0
rectal_temperature            0
pulse                         0
respiratory_rate              0
temperature_of_extremities    0
pain                          0
outcome                       0
dtype: int64

In [ ]:
ПС: А возможно представить данные в векторном виде и дополнить пустые ячейки на основе косинусного сходства векторов? 
Своего рода рекомендательная система

Хотел изначально пустые значения pain добавить каждой категории по одинаковому количеству, не сообразил как это сделать.
Буду рад если подскажите.